In [39]:
# Importações
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from google.colab import files
import numpy as np




In [ ]:
files.upload()

In [21]:
Tweets = pd.read_csv('Tweets.csv')
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [23]:
Tweets.groupby(['airline_sentiment']).size()

,0
airline_sentiment,
negative,9178
neutral,3099
positive,2363


In [25]:
Tweets = Tweets[Tweets['airline_sentiment_confidence'] > 0.8]

In [29]:
# tratamento do texto
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values) # Criando o modelo

In [32]:
X = token.texts_to_sequences(Tweets['text'].values) # gera os tokens
X = pad_sequences(X, padding="post", maxlen=100) # gera os vetores


In [33]:
print(X)

[[97 62  0 ...  0  0  0]
 [97 99  1 ...  0  0  0]
 [97  9 99 ...  0  0  0]
 ...
 [13 98 93 ...  0  0  0]
 [13 89  1 ...  0  0  0]
 [13  6 23 ...  0  0  0]]


In [35]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment'])
print(y)

[1 0 0 ... 0 1 0]


In [46]:
# passar cada sentimento para uma coluna diferente
y = to_categorical(y)
print(y[0:1])

[[[[[[[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]


     [[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]]



    [[[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]


     [[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]]]




   [[[[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]


     [[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]]



    [[[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]


     [[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]]]]





  [[[[[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]


     [[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]]



    [[[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]


     [[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]]]




   [[[[[0. 1.]
       [1. 0.]]

      [[1. 0.]
       [0. 1.]]]


     [[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       [1. 0.]]]]



    [[[[1. 0.]
       [0. 1.]]

      [[0. 1.]
       

In [51]:
# dividir as variaveis para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test[1:2]

array([[16, 15,  7, 25, 32, 75,  5, 24, 71,  3, 25, 25, 87, 41,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [60]:
# empilhar as camadas
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelo.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_4                  │ ?                           │     0 (unbuilt) │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
